In [1]:
%load_ext edx_jupyter

In [2]:
%%vertica
SELECT 'Connected to Vertica!'

password: ········


,?column?
0,Connected to Vertica!


Look at enrollees in self-paced courses.
Capture their engagement within the 1st n weeks of their enrollment.

In [8]:
%%vertica
DROP TABLE IF EXISTS self_paced_courses;
CREATE local TEMPORARY TABLE self_paced_courses
ON COMMIT PRESERVE ROWS AS
SELECT
    *
FROM 
    business_intelligence.course_master
WHERE
    is_active AND
    pacing_type = 'self_paced'

,course_number,course_id


In [13]:
%%vertica
DROP TABLE IF EXISTS enrollments;
CREATE local TEMPORARY TABLE enrollments
ON COMMIT PRESERVE ROWS AS
SELECT
    CASE
        WHEN c.course_start_date >= uc.first_enrollment_time THEN c.course_start_date
        WHEN c.course_start_date < uc.first_enrollment_time THEN DATE(uc.first_enrollment_time)
    END AS content_availability_date,
    uc.user_id,
    uc.course_id,
    uc.first_enrollment_time,
    uc.first_verified_enrollment_time
FROM
    production.d_user_course uc
JOIN
    self_paced_courses c
ON
    c.course_id = uc.course_id

,course_number,course_id


In [16]:
%%vertica -> results

SELECT
    SUM(engaged_in_first_week)/COUNT(*) AS 'engaged_in_first_week',
    SUM(engaged_in_second_week)/COUNT(*) AS 'engaged_in_second_week',
    SUM(engaged_in_third_week)/COUNT(*) AS 'engaged_in_third_week',
    COUNT(*) AS 'cnt_enrollments'
FROM
(
    SELECT
        bi_engage.user_id,
        bi_engage.course_id,
        CASE
            WHEN DATEDIFF('day', enr.content_availability_date, bi_engage.date) BETWEEN 0 AND 6 THEN 1
            ELSE 0
        END AS engaged_in_first_week,
        CASE
            WHEN DATEDIFF('day', enr.content_availability_date, bi_engage.date) BETWEEN 7 AND 13 THEN 1
            ELSE 0
        END AS engaged_in_second_week,
        CASE
            WHEN DATEDIFF('day', enr.content_availability_date, bi_engage.date) BETWEEN 14 AND 20 THEN 1
            ELSE 0
        END AS engaged_in_third_week
    FROM
        enrollments AS enr
    JOIN
        business_intelligence.activity_engagement_user_daily bi_engage
    ON
        enr.course_id = bi_engage.course_id AND
        enr.user_id = bi_engage.user_id
) AS enr

In [17]:
results

,engaged_in_first_week,engaged_in_second_week,engaged_in_third_week,cnt_enrollments
0,0.398557747663365043,0.113629749551179916,0.075533086703209386,10882423


engaged_in_first_week: 0.399, engaged_in_second_week: 0.114, engaged_in_third_week: 0.076

In [18]:
relative_increase_in_rate = 0.1
power = 0.8
significance_level = 0.1

In [19]:
! sudo pip install statsmodels

The directory '/home/developer/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/developer/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [20]:
import statsmodels.stats.api as sms

In [24]:
baseline_1st_week_rate, baseline_2nd_week_rate, baseline_3rd_week_rate = (
    float(results.loc[0].engaged_in_first_week),
    float(results.loc[0].engaged_in_second_week),
    float(results.loc[0].engaged_in_third_week)
)   

In [25]:
sample_sizes = []
for rate in (baseline_1st_week_rate, baseline_2nd_week_rate, baseline_3rd_week_rate):
    es = sms.proportion_effectsize(rate, rate * (1 + relative_increase_in_rate))
    size = round(sms.NormalIndPower().solve_power(es, power=power, alpha=significance_level, ratio=1))
    sample_sizes.append(size)

In [27]:
print(
    'It looks like we will need a sample size of either \n'
    '\t{} (1st week engagement), \n'
    '\t{} (2nd week engagement), or \n'
    '\t{} (3rd week engagement) \n'
    'depending on which week we want to track.'
    .format(sample_sizes[0], sample_sizes[1], sample_sizes[2])
)

It looks like we will need a sample size of either 
	1893 (1st week engagement), 
	10057 (2nd week engagement), or 
	15816 (3rd week engagement) 
depending on which week we want to track.
